In [1]:
import pandas as pd
import datetime 
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
%%time
smileon = pd.read_csv('../data/Smileon.2021.10.17.csv', skiprows=2)
smileon.head()

#NOTE TO SELF: to establish a branch: git checkout -b "name"

Wall time: 957 ms


C:\Users\Owner\anaconda3\lib\site-packages\decorator.py:231: DtypeWarning: Columns (2,32,36,53,58,59,77,83,89,93,105,127,141,144) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


,CallReportNum,ReportVersion,LinkedToCallReportNum,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
0,93115324,SMILE ON 60+ Registration,NaN,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,SMILE ON 60+ Registration,NaN,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,NaN,NaN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,SMILE ON 60+ Clinical,NaN,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,SMILE ON 60+ Clinical,NaN,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,SMILE ON 60+ Clinical,NaN,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN


In [3]:
smileon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 184 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(103), int64(6), object(75)
memory usage: 81.2+ MB


In [4]:
#dictionary says linedtocall was supposed to link call reports - is there anything in that column?
smileon.LinkedToCallReportNum.unique()

array([nan, 'Yes'], dtype=object)

In [5]:
smileon.loc[smileon['LinkedToCallReportNum']=='Yes']

,CallReportNum,ReportVersion,LinkedToCallReportNum,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
49990,61456060,SMILE ON 60+ Clinical,Yes,3/21/2019 13:24,3/21/2019 13:34,10,2652017,119475,NaN,NaN,...,NaN,NaN,Smiles Inc. - South Pittsburg - serving Marion...,Provider Site,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#looks like i can safely drop that column
#smileon = smileon.drop(['LinkedToCallReportNum'], axis=1)
#dropping columns later using MUCH more efficient code from teammate ... 

In [7]:
#i find the wording in the reportversion column annoying - is this an easy fix?
smileon.ReportVersion.unique()

array(['SMILE ON 60+ Registration', 'SMILE ON 60+ Clinical'], dtype=object)

In [8]:
smileon['ReportVersion'].replace('SMILE ON 60+ Registration', 'registration', inplace = True)
smileon['ReportVersion'].replace('SMILE ON 60+ Clinical', 'clinical', inplace = True)
smileon.ReportVersion.unique()

array(['registration', 'clinical'], dtype=object)

In [9]:
#let's rename some columns - an easy start
smileon = smileon.rename(columns={'CallDateAndTimeStart':'callstart', 'CallDateAndTimeEnd': 'callend', 'CallLength':'calllength', 'CallerNum':'patient_id', 'PhoneWorkerNum':'staff_id'})
smileon.head()

,CallReportNum,ReportVersion,LinkedToCallReportNum,callstart,callend,calllength,patient_id,staff_id,WasRealCall,WasHangup,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
0,93115324,registration,NaN,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,registration,NaN,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,NaN,NaN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,clinical,NaN,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,clinical,NaN,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,clinical,NaN,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN


In [10]:
#dictionary says the calltypes are unused.  is it true?
smileon.WasRealCall.unique()
smileon.WasHangup.unique()
smileon.WasSexCall.unique()
smileon.WasWrongNumber.unique()
smileon.WasPrankCall.unique()
smileon.WasSilentCall.unique()
smileon.GeoCode.unique()
smileon.GeoAssignment.unique()
#is true.  safe to assume that dictionary is accurate - drop all columns of "unused" value

array([nan])

In [11]:
smileon.drop(smileon.columns[[2,8, 9, 10, 11, 12, 13, 14, 15, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 49, 55, 57]], axis = 1, inplace = True)
smileon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 149 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(70), int64(6), object(73)
memory usage: 65.8+ MB


In [12]:
#149 columns ... getting better.  Some have notes about intended useage ... is useful?
smileon.PhoneType.unique()

array(['Mobile', nan, 'Home', 'Work', 'Other', 'Pager'], dtype=object)

In [13]:
#eh ... the question we're trying to answer is about effectivness of treatment.  Does this inform that?  I don't think so. 
smileon_edited = smileon.drop(columns=['PhoneType'])

In [14]:
smileon_edited.Narrative.unique()
#don't know that this is useful - but should leave it for now

array(['spoke with client again concerning transfer to IFD. Waiting to hear from clinic',
       'Called client to follow up on app. LM for callback. 3rd call.',
       nan, ...,
       'Verify insurance and update eligibility before proceeding .  HP',
       'Correct date of birth before proceeding with this patient.  Verify eligibility.  HP 4/29/2019',
       'Deleted duplicate registration created for this patient on 4/9/19.  Patient does not have dental insurance and is eligible for program.  HP 4/12/19.'],
      dtype=object)

In [15]:
smileon_edited.Feedback.unique()
#again, not sure this is useful - but leave for now?
#maybe not.  looks like a distraction for the scope of this project
#would be interesting to see if there is any followthrough on the narrative and feedback notes.  are changes being done?
#but for this project ... i don't think that's the q we're trying to answer.
#and i don't know how to begin answering that q without a systematic rowbyrow examination which ... no.
#also, a lot of it seems to be referring to data i don't have access to to check on if the changes were done or not.  
#so, not needed

array([nan,
       'Hi Rachel, this record is not attached to a patient. Please delete and reenter! Thanks, Mattie',
       '(No feedback was needed for this call.)',
       'Hi Victoria, Can you enter a registration for this patient? In the future, a registration should be created first before you enter any clinical entries. Please let me or Kim know if you have questions about that! Thanks!Mattie',
       'Hi Lawanna! Can you correct the DOB on this one? Thanks! - Mattie',
       "Hi Brenda!Lindsay has been trying to reach this patient about her appointment - if she contacts you back please give her Lindsay's info! It's jsut going straight to voicemail when Lindsay tries to contact her. Thanks! -Mattie",
       'Hi Jacinda! This lab assist code should be accompanied by the other initial denture code as well. Can you add that? Thanks! Mattie',
       'Hi Traci! This client was not registered in iCarol before you entered this form, so there is no profile. Can you please create a regist

In [16]:
smileon_edited = smileon_edited.drop(columns=['Narrative','Feedback'])

In [17]:
smileon_edited.EnteredByWorkerNum.unique()
#dictionary states that this column is to show who entered the data - may be different than who took the call.  is important? 
#might be important if there's a lot of that in one clinic.  but dictionary says is rare ... 
#could ask how rare? but seems like another distraction.  
#which is more important to us - who took the call or who entered the data?  for now, maybe keep both.  move column over when i finish deleting/renaming things
#let's rename some columns - an easy start
smileon_edited = smileon_edited.rename(columns={'EnteredByWorkerNum':'entrystaff_id','staff_id':'callstaff_id','EnteredOn':'entrydate'})

In [18]:
smileon_edited.info()
#down to 146 - keep going
smileonlist=list(smileon_edited)
print(smileonlist)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 146 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(70), int64(6), object(70)
memory usage: 64.4+ MB
['CallReportNum', 'ReportVersion', 'callstart', 'callend', 'calllength', 'patient_id', 'callstaff_id', 'CityName', 'CountyName', 'StateProvince', 'CountryName', 'PostalCode', 'entrystaff_id', 'entrydate', 'Reviewed', 'FeedbackStatus', 'FeedbackFromPhoneWorkerNum', 'FeedbackFromPhoneWorkerName', 'ReferralsMade', 'OrgNum', 'SMILE ON 60+ Demographic Information - # of People in Household', 'Birth Year', 'SMILE ON 60+ Demographic Information - Other Language', 'SMILE ON 60+ Notes (Optional) - Notes', 'SMILE ON 60+ Oral Care Encounter - 1351 - Quantity', 'SMILE ON 60+ Oral Care Encounter - 1354 - Quantity', 'SMILE ON 60+ Oral Care Encounter - 2140 - Quantity', 'SMILE ON 60+ Oral Care Encounter - 2150 - Quantity', 'SMILE ON 60+ Oral Care Encount

In [19]:
smileon_edited.Reviewed.unique()
smileon_edited.FeedbackStatus.unique()
smileon_edited = smileon_edited.drop(columns=['FeedbackStatus','FeedbackFromPhoneWorkerNum','FeedbackFromPhoneWorkerName','OrgNum'])
smileon_edited.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 142 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(69), int64(5), object(68)
memory usage: 62.7+ MB


In [20]:
smileon_edited.ReferralsMade.unique()
#smileon_edited['SMILE ON 60+ Demographic Information-# of People in Household'].unique()
#okay - so, that's not working.  maybe need to rename columns to even see what's in them ... 

array([nan, 'Neighborhood Health - Cleveland Park; ',
       'Tennessee Department of Health - Maury County; ',
       'Neighborhood Health - Lebanon ; ',
       'Church Health Center - Memphis; ',
       'Interfaith Dental Clinic of Rutherford County; ',
       'Reelfoot Ministries - Obion; ',
       'Church Health Center - Memphis; Christ Community Health Services - Raleigh; ',
       'Smiles Inc. - South Pittsburg; ',
       'Interfaith Health Clinic -Knoxville; ',
       'Interfaith Dental Clinic - Murfreesboro; ',
       'Interfaith Dental Clinic - Nashville; ',
       'Mountain Hope Good Shepherd Clinic; ',
       'Matthew Walker Comprehensive Health Center - Nashville; ',
       'Christ Community Health Services - Third Street; ',
       'Christ Community Health Services - Hickory Hill; ',
       'Christ Community Health Services - Frayser; ',
       'Karis Dental Clinic-Cleveland ; ',
       'Christ Community Health Services - Broad Avenue; ',
       'Interfaith Dental Clinic -

In [21]:
smileon_edited = smileon_edited.rename(columns={'SMILE ON 60+ Demographic Information - # of People in Household':'demographic_household', 'SMILE ON 60+ Demographic Information - Other Language':'demographic_lang'})

In [22]:
smileon_edited.demographic_household.unique()
#well, that's a hot mess.  if i need to use this column, i'll need to clean it first.  not sure if i need it though ... 
#demographic_lang is pretty straightforward.  looks potentially insightful

array([1.0, 2.0, 6.0, 4.0, 7.0, 3.0, 8.0, 5.0, nan, 10.0, 9.0, '1', '3',
       '2', '4', '5', '6', '7', '9', '8', '1_____', '3_____', '0', '112',
       '2864', '2_____', '1886', '4_____', '8368', '13900', '______',
       '112___', '6_____'], dtype=object)

In [23]:
#okay - these remaining column names need some love.  let's change them to be useful. 
#probably, definitely, a better way to do this, but i know that this works, and is clear:
smileon_edited = smileon_edited.rename(columns={
    'SMILE ON 60+ Notes (Optional) - Notes':'notes',
    'SMILE ON 60+ Oral Care Encounter - 1351 - Quantity':'oce_1351',
    'SMILE ON 60+ Oral Care Encounter - 1354 - Quantity':'oce_1354',
    'SMILE ON 60+ Oral Care Encounter - 2140 - Quantity':'oce_2140',
    'SMILE ON 60+ Oral Care Encounter - 2150 - Quantity':'oce_2150',
    'SMILE ON 60+ Oral Care Encounter - 2160 - Quantity':'oce_2160',
    'SMILE ON 60+ Oral Care Encounter - 2161 - Quantity':'oce_2161',
    'SMILE ON 60+ Oral Care Encounter - 220 - Quantity':'oce_220',
    'SMILE ON 60+ Oral Care Encounter - 230 - Quantity':'oce_230',
    'SMILE ON 60+ Oral Care Encounter - 2330 - Quantity':'oce_2330',
    'SMILE ON 60+ Oral Care Encounter - 2331 - Quantity':'oce_2331',
    'SMILE ON 60+ Oral Care Encounter - 2332 - Quantity':'oce_2332',
    'SMILE ON 60+ Oral Care Encounter - 2335 - Quantity':'oce_2335',
    'SMILE ON 60+ Oral Care Encounter - 2390 - Quantity':'oce_2390',
    'SMILE ON 60+ Oral Care Encounter - 2391 - Quantity':'oce_2391',
    'SMILE ON 60+ Oral Care Encounter - 2392 - Quantity':'oce_2392',
    'SMILE ON 60+ Oral Care Encounter - 2393 - Quantity':'oce_2393',
    'SMILE ON 60+ Oral Care Encounter - 2394 - Quantity':'oce_2394',
    'SMILE ON 60+ Oral Care Encounter - 2915 - Quantity':'oce_2915',
    'SMILE ON 60+ Oral Care Encounter - 2920 - Quantity':'oce_2920',
    'SMILE ON 60+ Oral Care Encounter - 2933 - Quantity':'oce_2933',
    'SMILE ON 60+ Oral Care Encounter - 2940 - Quantity':'oce_2940',
    'SMILE ON 60+ Oral Care Encounter - 2950 - Quantity':'oce_2950',
    'SMILE ON 60+ Oral Care Encounter - 2954 - Quantity':'oce_2954',
    'SMILE ON 60+ Oral Care Encounter - 2955 - Quantity':'oce_2955',
    'SMILE ON 60+ Oral Care Encounter - 2980 - Quantity':'oce_2980',
    'SMILE ON 60+ Oral Care Encounter - 3220 - Quantity':'oce_3220',
    'SMILE ON 60+ Oral Care Encounter - 4341 - Quantity':'oce_4341',
    'SMILE ON 60+ Oral Care Encounter - 4342 - Quantity':'oce_4342',
    'SMILE ON 60+ Oral Care Encounter - 6930 - Quantity':'oce_6930',
    'SMILE ON 60+ Oral Care Encounter - 6980 - Quantity':'oce_6980',
    'SMILE ON 60+ Oral Care Encounter - 7111 - Quantity':'oce_7111',
    'SMILE ON 60+ Oral Care Encounter - 7140 - Quantity':'oce_7140',
    'SMILE ON 60+ Oral Care Encounter - 7210 - Quantity':'oce_7210',
    'SMILE ON 60+ Oral Care Encounter - 7220 - Quantity':'oce_7220',
    'SMILE ON 60+ Oral Care Encounter - 7230 - Quantity':'oce_7230',
    'SMILE ON 60+ Oral Care Encounter - 7240 - Quantity':'oce_7240',
    'SMILE ON 60+ Oral Care Encounter - 7241 - Quantity':'oce_7241',
    'SMILE ON 60+ Oral Care Encounter - 7250 - Quantity':'oce_7250',
    'SMILE ON 60+ Oral Care Encounter - 7261 - Quantity':'oce_7261',
    'SMILE ON 60+ Oral Care Encounter - 7285 - Quantity':'oce_7285',
    'SMILE ON 60+ Oral Care Encounter - 7286 - Quantity':'oce_7286',
    'SMILE ON 60+ Oral Care Encounter - 7310 - Quantity':'oce_7310',
    'SMILE ON 60+ Oral Care Encounter - 7311 - Quantity':'oce_7311',
    'SMILE ON 60+ Oral Care Encounter - 7320 - Quantity':'oce_7320',
    'SMILE ON 60+ Oral Care Encounter - 7321 - Quantity':'oce_7321',
    'SMILE ON 60+ Oral Care Encounter - 7410 - Quantity':'oce_7410',
    'SMILE ON 60+ Oral Care Encounter - 7411 - Quantity':'oce_7411',
    'SMILE ON 60+ Oral Care Encounter - 7412 - Quantity':'oce_7412',
    'SMILE ON 60+ Oral Care Encounter - 7413  - Quantity':'oce_7413',
    'SMILE ON 60+ Oral Care Encounter - 7414 - Quantity':'oce_7414',
    'SMILE ON 60+ Oral Care Encounter - 7415 - Quantity':'oce_7415',
    'SMILE ON 60+ Oral Care Encounter - 7440 - Quantity':'oce_7440',
    'SMILE ON 60+ Oral Care Encounter - 7450 - Quantity':'oce_7450',
    'SMILE ON 60+ Oral Care Encounter - 7451 - Quantity':'oce_7451',
    'SMILE ON 60+ Oral Care Encounter - 7460 - Quantity':'oce_7460',
    'SMILE ON 60+ Oral Care Encounter - 7461 - Quantity':'oce_7461',
    'SMILE ON 60+ Oral Care Encounter - 7471 - Quantity':'oce_7471',
    'SMILE ON 60+ Oral Care Encounter - 7510 - Quantity':'oce_7510',
    'SMILE ON 60+ Oral Care Encounter - 7511 - Quantity':'oce_7511',
    'SMILE ON 60+ Oral Care Encounter - 7970 - Mandibular Quantity':'oce_7970_mandibular',
    'SMILE ON 60+ Oral Care Encounter - 7970 - Maxillary Quantity ':'oce_7970_maxillary',
    'SMILE ON 60+ Oral Care Encounter - 7971 - Quantity':'oce_7971',
    'SMILE ON 60+ Oral Care Encounter - 7972 - Quantity':'oce_7972',
    'SMILE ON 60+ Oral Care Encounter - 9110 - Quantity':'oce_9110',
    'SMILE ON 60+ Oral Care Encounter - 9951 - Quantity':'oce_9951',
    'SMILE ON 60+ Oral Care Encounter - Lab Assist narrative':'oce_labnarrative',
    'SMILE ON 60+ Oral Care Encounter - No Code - Denture Step Mandibular Quantity':'oce_0000_denturemandibular',
    'SMILE ON 60+ Oral Care Encounter - No Code - Denture Step Maxillary Quantity':'oce_0000_denturemaxillary',
    'SMILE ON 60+ Screening - Last Screening Date':'s_lastscreening_date',
    'SMILE ON 60+ Verification of Eligibility - Verification Date':'ve_verification_date',
    'SMILE ON 60+ Base-line Oral and Conditions Questions - Are you limited in what you can eat?':'blq_troubleeating',
    'SMILE ON 60+ Base-line Oral and Conditions Questions - Dental Clinic':'blq_dentalclinic',
    'SMILE ON 60+ Base-line Oral and Conditions Questions - Did the client commit to referral?':'blq_patientreferralcommitment',
    'SMILE ON 60+ Base-line Oral and Conditions Questions - Did you help in these other areas?':'blq_did_needhelp',
    'SMILE ON 60+ Base-line Oral and Conditions Questions - Do you need help in these other areas?':'blq_do_needhelp',
    'SMILE ON 60+ Base-line Oral and Conditions Questions - Have you ever felt that the appearance of your mouth and teeth affected your quality of life?':'blq_qualityoflife',
    'SMILE ON 60+ Base-line Oral and Conditions Questions - How would you describe the condition of your mouth and teeth?':'blq_mouthcondition',
    'SMILE ON 60+ Base-line Oral and Conditions Questions - What is the next step for client encounter?':'blq_nextstep',
    'SMILE ON 60+ Base-line Oral and Conditions Questions - What is the next step for clients encounter?':'blq_nextstep2',
    'SMILE ON 60+ Case Management Information - Dental Clinic':'cmi_clinic',
    'SMILE ON 60+ Case Management Information - Transportation Barriers':'cmi_transitproblem',
    'SMILE ON 60+ Case Management Information - What barriers have kept you from finding a consistent dental home? (Select all that apply)':'cmi_findingcareproblem',
    'SMILE ON 60+ Case Management Information - What is the outcome for the client?':'cmi_clientoutcome',
    'SMILE ON 60+ Clinic Information - Action: Upload Before Picture on Finish Tab':'ci_beforephoto',
    'SMILE ON 60+ Clinic Information - Is patient of record date if yes last seen?':'ci_lastseen_date',
    'SMILE ON 60+ Clinic Information - Was initial appointment made?':'ci_apptmade',
    'SMILE ON 60+ Clinic Information - Was initial transportation assistance or referral provided?':'ci_transitprovided',
    'SMILE ON 60+ Demographic Information - Gender':'demographic_gender',
    'SMILE ON 60+ Demographic Information - Household Income Level':'demographic_income',
    'SMILE ON 60+ Demographic Information - How did you first hear about the SMILE ON 60+ program?':'demographic_howyouhearprogram',
    'SMILE ON 60+ Demographic Information - Preferred Language/Language Needed':'demographic_preferredlang',
    'SMILE ON 60+ Demographic Information - Race/Ethnicity':'demographic_ethnic',
    'SMILE ON 60+ Demographic Information - What other assistance programs are you on?':'demographic_otherassistance',
    'SMILE ON 60+ Demographic Information - When is the last time you visited your dentist?':'demographic_lastdentistvisit',
    'SMILE ON 60+ Oral Care Encounter - Action: Upload After Picture on Finish Tab':'oce_afterphoto',
    'SMILE ON 60+ Oral Care Encounter - Is the treatment plan completed?':'oce_treatmentplan_complete',
    'SMILE ON 60+ Oral Care Encounter - Was a treatment plan developed?':'oce_treatmentplan_developed',
    'SMILE ON 60+ Oral Care Encounter - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)':'oce_clientrestored',
    'SMILE ON 60+ Oral Care Encounter - What care was provided to the enrollee? (Select all that apply)':'oce_WHATCAREGIVEN',
    'SMILE ON 60+ Presumptive Eligibility - Dental Insurance?':'pe_insurance',
    'SMILE ON 60+ Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?':'pe_smileonprequalify',
    'SMILE ON 60+ Presumptive Eligibility - What is the next step for clients encounter? ':'pe_nextstep',
    'SMILE ON 60+ Presumptive Eligibility - What is the next step for clients encounter?':'pe_nextstep2',
    'SMILE ON 60+ Screening - # of Lower Natural Teeth':'s_num_lowerteeth',
    'SMILE ON 60+ Screening - # of Upper Natural Teeth':'s_num_upperteeth',
    'SMILE ON 60+ Screening - Do you have removable dentures?':'s_dentures',
    'SMILE ON 60+ Screening - Do you usually wear your denture(s) when you eat?':'s_dentureswhileeating',
    'SMILE ON 60+ Screening - Experienced any problems related to mouth/teeth during past 12 months?':'s_problems12months',
    'SMILE ON 60+ Screening - How often do you brush your teeth?':'s_brushing',
    'SMILE ON 60+ Screening - Need for Periodontal Care':'s_needforcare',
    'SMILE ON 60+ Screening - Root Fragments':'s_rootfrags',
    'SMILE ON 60+ Screening - Suspicious Soft Tissue Lesions':'s_lesions',
    'SMILE ON 60+ Screening - Treatment Urgency':'s_urgency',
    'SMILE ON 60+ Screening - Untreated Decay':'s_decay',
    'SMILE ON 60+ Site Information - Enroller ID':'si_clinicname',
    'SMILE ON 60+ Site Information - Enroller Location':'si_typewhereclientenrolled',
    'SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab':'tp_afterphoto',
    'SMILE ON 60+ Treatment Plan - Is the treatment plan completed?':'tp_plancomplete',
    'SMILE ON 60+ Treatment Plan - Was a treatment plan developed?':'tp_plandeveloped',
    'SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)':'tp_clientrestored',
    'SMILE ON 60+ Verification of Eligibility - Transportation Arranged':'ve_transitarranged',
    'SMILE ON 60+ Verification of Eligibility - Verification of Eligibility':'ve_eligibility'
    })

In [24]:
smileonlist2=list(smileon_edited)
print(smileonlist2)

['CallReportNum', 'ReportVersion', 'callstart', 'callend', 'calllength', 'patient_id', 'callstaff_id', 'CityName', 'CountyName', 'StateProvince', 'CountryName', 'PostalCode', 'entrystaff_id', 'entrydate', 'Reviewed', 'ReferralsMade', 'demographic_household', 'Birth Year', 'demographic_lang', 'notes', 'oce_1351', 'oce_1354', 'oce_2140', 'oce_2150', 'oce_2160', 'oce_2161', 'oce_220', 'oce_230', 'oce_2330', 'oce_2331', 'oce_2332', 'oce_2335', 'oce_2390', 'oce_2391', 'oce_2392', 'oce_2393', 'oce_2394', 'oce_2915', 'oce_2920', 'oce_2933', 'oce_2940', 'oce_2950', 'oce_2954', 'oce_2955', 'oce_2980', 'oce_3220', 'oce_4341', 'oce_4342', 'oce_6930', 'oce_6980', 'oce_7111', 'oce_7140', 'oce_7210', 'oce_7220', 'oce_7230', 'oce_7240', 'oce_7241', 'oce_7250', 'oce_7261', 'oce_7285', 'oce_7286', 'oce_7310', 'oce_7311', 'oce_7320', 'oce_7321', 'oce_7410', 'oce_7411', 'oce_7412', 'oce_7413', 'oce_7414', 'oce_7415', 'oce_7440', 'oce_7450', 'oce_7451', 'oce_7460', 'oce_7461', 'oce_7471', 'oce_7510', 'oce

In [25]:
#okay! those photo columns - anything there?
smileon_edited.tp_afterphoto.unique()
smileon_edited.oce_labnarrative.unique() #more notes - not sure i need this
smileon_edited.ci_beforephoto.unique()
smileon_edited.oce_afterphoto.unique()

array([nan])

In [26]:
smileon_edited=smileon_edited.drop(columns=['tp_afterphoto','oce_labnarrative','ci_beforephoto','oce_afterphoto'])

In [27]:
smileon_edited.StateProvince.unique()
smileon_edited.CountryName.unique()

array(['United States', nan], dtype=object)

In [28]:
smileon_edited=smileon_edited.drop(columns=['CountryName'])

In [29]:
#rename those last couple :
smileon_edited = smileon_edited.rename(columns={
    'CallReportNum':'callreportnum',
    'ReportVersion':'reportversion',
    'CityName':'city',
    'CountyName':'county',
    'StateProvince':'state',
    'PostalCode':'zip',
    'Birth Year':'patient_bday'
})

In [30]:
smileon_edited.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 137 entries, callreportnum to ve_eligibility
dtypes: float64(68), int64(5), object(64)
memory usage: 60.5+ MB


In [31]:
smileonlist3=list(smileon_edited)
print(smileonlist3)

['callreportnum', 'reportversion', 'callstart', 'callend', 'calllength', 'patient_id', 'callstaff_id', 'city', 'county', 'state', 'zip', 'entrystaff_id', 'entrydate', 'Reviewed', 'ReferralsMade', 'demographic_household', 'patient_bday', 'demographic_lang', 'notes', 'oce_1351', 'oce_1354', 'oce_2140', 'oce_2150', 'oce_2160', 'oce_2161', 'oce_220', 'oce_230', 'oce_2330', 'oce_2331', 'oce_2332', 'oce_2335', 'oce_2390', 'oce_2391', 'oce_2392', 'oce_2393', 'oce_2394', 'oce_2915', 'oce_2920', 'oce_2933', 'oce_2940', 'oce_2950', 'oce_2954', 'oce_2955', 'oce_2980', 'oce_3220', 'oce_4341', 'oce_4342', 'oce_6930', 'oce_6980', 'oce_7111', 'oce_7140', 'oce_7210', 'oce_7220', 'oce_7230', 'oce_7240', 'oce_7241', 'oce_7250', 'oce_7261', 'oce_7285', 'oce_7286', 'oce_7310', 'oce_7311', 'oce_7320', 'oce_7321', 'oce_7410', 'oce_7411', 'oce_7412', 'oce_7413', 'oce_7414', 'oce_7415', 'oce_7440', 'oce_7450', 'oce_7451', 'oce_7460', 'oce_7461', 'oce_7471', 'oce_7510', 'oce_7511', 'oce_7970_mandibular', 'oce_

In [32]:
#let's try and reorganize these columns so the kinds are together
#note - i tested this first to make sure i didn't drop anything unintentionally

smileon_edited = smileon_edited[['patient_id','callreportnum', 'reportversion', 'callstart', 'callend', 'calllength',
                             'callstaff_id','entrystaff_id', 'entrydate', 'Reviewed', 'ReferralsMade',
                             'patient_bday', 'city', 'county', 'state', 'zip', 
                             'demographic_household','demographic_lang','demographic_preferredlang', 'demographic_ethnic','demographic_gender', 'demographic_income', 'demographic_howyouhearprogram', 'demographic_otherassistance', 'demographic_lastdentistvisit',
                             'notes',
                             'pe_insurance', 'pe_smileonprequalify', 'pe_nextstep', 'pe_nextstep2',
                             've_transitarranged', 've_eligibility','ve_verification_date',
                             'si_clinicname', 'si_typewhereclientenrolled', 
                             'blq_troubleeating', 'blq_dentalclinic', 'blq_patientreferralcommitment', 'blq_did_needhelp', 'blq_do_needhelp', 'blq_qualityoflife', 'blq_mouthcondition', 'blq_nextstep', 'blq_nextstep2',
                             's_num_lowerteeth', 's_num_upperteeth', 's_dentures', 's_dentureswhileeating', 's_problems12months', 's_brushing', 's_needforcare', 's_rootfrags', 's_lesions', 's_urgency', 's_decay','s_lastscreening_date',
                             'ci_lastseen_date', 'ci_apptmade', 'ci_transitprovided', 
                             'cmi_clinic', 'cmi_transitproblem', 'cmi_findingcareproblem', 'cmi_clientoutcome',
                             'oce_220', 'oce_230','oce_1351', 'oce_1354', 'oce_2140', 'oce_2150', 'oce_2160', 'oce_2161', 'oce_2330', 'oce_2331', 'oce_2332', 'oce_2335', 'oce_2390', 'oce_2391', 'oce_2392', 'oce_2393', 'oce_2394', 'oce_2915', 'oce_2920', 'oce_2933', 'oce_2940', 'oce_2950', 'oce_2954', 'oce_2955', 'oce_2980', 'oce_3220', 'oce_4341', 'oce_4342', 'oce_6930', 'oce_6980', 'oce_7111', 'oce_7140', 'oce_7210', 'oce_7220', 'oce_7230', 'oce_7240', 'oce_7241', 'oce_7250', 'oce_7261', 'oce_7285', 'oce_7286', 'oce_7310', 'oce_7311', 'oce_7320', 'oce_7321', 'oce_7410', 'oce_7411', 'oce_7412', 'oce_7413', 'oce_7414', 'oce_7415', 'oce_7440', 'oce_7450', 'oce_7451', 'oce_7460', 'oce_7461', 'oce_7471', 'oce_7510', 'oce_7511', 'oce_7970_mandibular', 'oce_7970_maxillary', 'oce_7971', 'oce_7972', 'oce_9110', 'oce_9951', 'oce_0000_denturemandibular', 'oce_0000_denturemaxillary', 
                             'oce_treatmentplan_developed','oce_treatmentplan_complete', 'oce_clientrestored', 'oce_WHATCAREGIVEN', 
                             'tp_plancomplete', 'tp_plandeveloped', 'tp_clientrestored'
                            ]]

In [33]:
smileon_edited.notes.unique()
smileon_edited = smileon_edited.drop(columns=['notes'])
smileon_edited.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 136 entries, patient_id to tp_clientrestored
dtypes: float64(68), int64(5), object(63)
memory usage: 60.0+ MB


In [40]:
smileonlist3=list(smileon_edited)
print(smileonlist3)

['patient_id', 'callreportnum', 'reportversion', 'callstart', 'callend', 'calllength', 'callstaff_id', 'entrystaff_id', 'entrydate', 'Reviewed', 'ReferralsMade', 'patient_bday', 'city', 'county', 'state', 'zip', 'demographic_household', 'demographic_lang', 'demographic_preferredlang', 'demographic_ethnic', 'demographic_gender', 'demographic_income', 'demographic_howyouhearprogram', 'demographic_otherassistance', 'demographic_lastdentistvisit', 'pe_insurance', 'pe_smileonprequalify', 'pe_nextstep', 'pe_nextstep2', 've_transitarranged', 've_eligibility', 've_verification_date', 'si_clinicname', 'si_typewhereclientenrolled', 'blq_troubleeating', 'blq_dentalclinic', 'blq_patientreferralcommitment', 'blq_did_needhelp', 'blq_do_needhelp', 'blq_qualityoflife', 'blq_mouthcondition', 'blq_nextstep', 'blq_nextstep2', 's_num_lowerteeth', 's_num_upperteeth', 's_dentures', 's_dentureswhileeating', 's_problems12months', 's_brushing', 's_needforcare', 's_rootfrags', 's_lesions', 's_urgency', 's_decay

In [35]:
#i want to see all my columns
pd.set_option('display.max_columns', 137)
smileon_edited.describe()

,patient_id,callreportnum,calllength,callstaff_id,entrystaff_id,zip,s_num_lowerteeth,s_num_upperteeth,oce_220,oce_230,oce_1351,oce_1354,oce_2140,oce_2150,oce_2160,oce_2161,oce_2330,oce_2331,oce_2332,oce_2335,oce_2390,oce_2391,oce_2392,oce_2394,oce_2915,oce_2920,oce_2933,oce_2940,oce_2954,oce_2955,oce_2980,oce_3220,oce_4341,oce_6930,oce_6980,oce_7111,oce_7210,oce_7220,oce_7230,oce_7240,oce_7241,oce_7250,oce_7261,oce_7285,oce_7286,oce_7310,oce_7311,oce_7321,oce_7410,oce_7411,oce_7412,oce_7413,oce_7414,oce_7415,oce_7440,oce_7450,oce_7451,oce_7460,oce_7461,oce_7471,oce_7510,oce_7511,oce_7970_mandibular,oce_7970_maxillary,oce_7971,oce_7972,oce_9110,oce_9951,oce_0000_denturemandibular,oce_0000_denturemaxillary,oce_treatmentplan_developed,oce_treatmentplan_complete,oce_clientrestored
count,5.786000e+04,5.786000e+04,57860.000000,57860.000000,57860.000000,54791.000000,6238.000000,5352.000000,0.0,1337.000000,12.000000,892.000000,50.000000,104.000000,46.000000,10.0,944.000000,940.000000,579.000000,428.000000,8.0,1226.000000,1341.000000,171.000000,2.0,134.000000,19.000000,74.000000,15.0,0.0,3.0,7.0,1396.000000,12.000000,2.0,9.000000,819.000000,5.000000,6.000000,19.000000,3.000000,486.000000,1.0,0.0,68.000000,225.000000,30.000000,80.000000,16.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,12.0,0.0,0.0,0.0,2.0,1.0,26.000000,83.0,0.0,0.0,0.0,0.0,0.0
mean,3.080123e+06,7.417346e+07,4.096319,124757.402402,124847.296906,37551.859229,10.102597,9.840060,NaN,2.549738,2.416667,3.242152,1.280000,1.057692,1.043478,1.0,1.507415,1.361702,1.255613,1.343458,1.0,1.273246,1.158091,1.116959,1.0,1.037313,1.052632,1.108108,1.0,NaN,1.0,1.0,1.832378,1.083333,1.0,1.777778,1.800977,1.200000,1.166667,3.000000,3.666667,2.425926,1.0,NaN,1.073529,1.853333,1.233333,1.237500,2.187500,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,1.0,NaN,NaN,NaN,1.0,1.0,1.038462,1.0,NaN,NaN,NaN,NaN,NaN
std,5.901309e+05,1.086493e+07,10.495227,14700.697026,14244.598657,507.518188,3.184517,3.539749,NaN,2.173077,2.353270,3.038418,1.050559,0.234290,0.206185,0.0,1.027691,1.062990,0.628297,0.769001,0.0,0.579558,0.524394,0.403374,0.0,0.190240,0.229416,0.424186,0.0,NaN,0.0,0.0,0.814200,0.288675,0.0,1.394433,2.092763,0.447214,0.408248,4.558265,3.785939,2.378960,NaN,NaN,0.262944,0.931013,0.568321,0.483755,1.833712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,0.0,NaN,0.196116,0.0,NaN,NaN,NaN,NaN,NaN
min,-2.000000e+00,5.319245e+07,0.000000,-1.000000,60132.000000,24201.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,NaN,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.000000,1.0,NaN,NaN,NaN,NaN,NaN
25%,2.633308e+06,6.541199e+07,1.000000,118845.000000,118845.000000,37146.000000,8.000000,7.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,NaN,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.500000,1.000000,1.0,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.000000,1.0,NaN,NaN,NaN,NaN,NaN
50%,2.891609e+06,7.156906e+07,1.000000,123747.000000,123747.000000,37334.000000,10.000000,10.000000,NaN,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,NaN,1.0,1.0,2.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.0,NaN,1.000000,2.000000,1.000000,1.000000,1.500000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.000000,1.0,NaN,NaN,NaN,NaN,NaN
75%,3.496070e+06,8.423162e+07,3.000000,134546.000000,134546.000000,38018.0000

In [36]:
smileon_edited['oce_clientrestored'].value_counts()

Series([], Name: oce_clientrestored, dtype: int64)

In [37]:
#i still have some columns with no entries/no data - let's drop those
smileon_edited = smileon_edited.drop(['oce_220','oce_2955', 'oce_7285', 'oce_7412', 'oce_7413','oce_7414','oce_7415', 'oce_7440','oce_7450','oce_7451','oce_7460','oce_7461','oce_7511','oce_7970_mandibular','oce_7970_maxillary','oce_0000_denturemandibular','oce_0000_denturemaxillary','oce_treatmentplan_developed','oce_treatmentplan_complete','oce_clientrestored'], axis=1)

In [38]:
smileon_edited.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 116 entries, patient_id to tp_clientrestored
dtypes: float64(48), int64(5), object(63)
memory usage: 51.2+ MB


In [39]:
smileon_edited.to_csv('../data/smileon_edited.csv')